<a href="https://colab.research.google.com/github/koklengyeo/aviation/blob/main/aviation_business_air_travel_v2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREAMBLE

In [27]:
#FOR FUTURE REFERENCE
#DOWNGRADE PYTHON TO VERSION USED DURING DEVELOPMENT
#!apt-get update -y
#!apt-get install python3.10 python3.10-distutils
#!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
#!update-alternatives --config python3
#!apt-get install python3-pip
#!python3 -m pip install --upgrade pip --user

In [28]:
#FOR FUTURE REFERENCE
#DOWNGRADE PACKAGES TO VERSION USED DURING DEVELOPMENT
#!pip install IPython==7.34.0
#!pip install ipywidgets==7.7.1
#!pip install matplotlib==3.7.1
#!pip install numpy==1.25.2
#!pip install pandas==2.0.3

In [29]:
#IMPORT PACKAGES
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import *
from ipywidgets import *

# USER INPUT FORM (WIDGETS)

In [30]:
#DEFINE WIDGETS USED TO CAPTURE USER INPUTS

#BASE YEAR
#MUST BE BETWEEN 2015 AND PRESENT YEAR
by_widget=widgets.BoundedIntText(
    value=datetime.now().year,
    min=2015,
    max=datetime.now().year,
    step=1,
    description="Base Year",
    disabled=False)

#TARGET YEAR
#MUST BE BETWEEN (PRESENT YEAR+5) AND 2050
ty_widget=widgets.BoundedIntText(
    value=datetime.now().year+5,
    min=datetime.now().year+5,
    max=2050,
    step=1,
    description="Target Year",
    disabled=False)

#BUSINESS AIR TRAVEL EMISSIONS IN TCO2E
#MUST BE >=0
bte_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Business Air Travel Emissions in tCO2e",
    disabled=False)

#BASE YEAR TOTAL FLOWN CONTRACTED FREIGHT IN RTK
#MUST BE >=0
by_bta_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Company FTE",
    disabled=False)

#TARGET YEAR TOTAL FLOWN CONTRACTED FREIGHT IN RTK
#MUST BE >=0
ty_bta_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Company FTE",
    disabled=False)

#INVISIBLE BOOLEAN WIDGET TO REGISTER FIRST CLICK OF "CALCULATE TARGET" BUTTON
fc_widget=widgets.Valid(
    value=False,
    layout={"display":"none"})

In [31]:
#DEFINE WIDGETS USED TO DISPLAY TEXT

overall_header=widgets.HTML(
    value="<h1>TARGET SETTING FOR BUSINESS AIR TRAVEL</h1>\
    <h3>Version 2.0 (Feb-23)</h3>This is intended for companies engaging in \
    business air travel to set targets on scope 3 category 6 (Business Travel) \
    emissions. Targets are aligned with a well below 2&deg;C scenario.")
basic_info_header=widgets.HTML(
    value="<h2>1. Basic Information</h2>Please indicate the base year and \
    target year. The base year should be the most recent year with a complete \
    GHG inventory. Target must cover a minimum of 5 years and a maximum of 10 \
    years from the year it is submitted to the SBTi for validation.")
by_emissions_header=widgets.HTML(
    value="<h2>2. Base Year Emissions</h2>")
by_activity_header=widgets.HTML(
    value="<h2>3. Base Year Activity</h2>Please provide the company workforce \
    in full-time equivalent (FTE) in the base year.")
ty_activity_header=widgets.HTML(
    value="<h2>4. Target Year Activity</h2>Please provide the expected company \
    FTE in the target year. The target model will assume the company FTE \
    changes at the constant CAGR that will bring it to this level in the \
    target year.")
ty_activity_footer=widgets.HTML(
    value="Please ensure that all provided information is as complete and \
    accurate as possible. Click the button below to calculate the target.")
results_header=widgets.HTML(
    value="<h2>5. Target Modelling Results</h2>")

#ERROR MESSAGES IF USER INPUTS DO NOT MEET CERTAIN CONDITIONS
no_error=widgets.HTML()
bte_error=widgets.HTML(
    value="<p style='color:red;'>Business Air Travel Emissions in tCO2e must \
      be provided.</p>")
bta_error=widgets.HTML(
    value="<p style='color:red;'>Company FTE must be provided.</p>")
user_inputs_error=widgets.HTML(
    value="<p style='color:red;'>Information provided incomplete and/or \
      invalid. Please check and try again.</p>")

In [32]:
#WIDGET LAYOUT & STYLE

#SELECTION WIDGETS
layout={"width":"1100px"}
style={"description_width":"600px"}

#YEAR WIDGETS
layout={"width":"700px"}
style={"description_width":"600px"}
by_widget.layout=layout
by_widget.style=style
ty_widget.layout=layout
ty_widget.style=style

#EMISSIONS WIDGETS
layout={"width":"800px"}
style={"description_width":"600px"}
bte_widget.layout=layout
bte_widget.style=style

#ACTIVITY WIDGETS
layout={"width":"800px"}
style={"description_width":"600px"}
by_bta_widget.layout=layout
by_bta_widget.style=style
ty_bta_widget.layout=layout
ty_bta_widget.style=style

#TEXT WIDGETS
layout={"width":"1100px"}
overall_header.layout=layout
basic_info_header.layout=layout
by_emissions_header.layout=layout
by_activity_header.layout=layout
ty_activity_header.layout=layout
ty_activity_footer.layout=layout
results_header.layout=layout
bte_error.layout=layout
bta_error.layout=layout
user_inputs_error.layout=layout

# USER INPUT FORM (INTERACTIVITY)

In [33]:
#USER INPUT FORM - BASIC INFORMATION

def basic_info_check(by,
                     ty,
                     fc):
  return True
basic_info_input=interactive(
    basic_info_check,
    by=by_widget,
    ty=ty_widget,
    fc=fc_widget)

In [34]:
#USER INPUT FORM - BASE YEAR EMISSIONS

def by_emissions_check(bte,
                       fc):
  by_emissions_valid=True
  update_display(no_error,display_id="by_emissions_check")
  #BTE - MUST BE PROVIDED
  if bte==0:
    by_emissions_valid=False
    if fc:
      display(bte_error,display_id="by_emissions_check")
  return by_emissions_valid
by_emissions_input=interactive(
    by_emissions_check,
    bte=bte_widget,
    fc=fc_widget)

In [35]:
#USER INPUT FORM - BASE YEAR ACTIVITY

def by_activity_check(by_bta,
                      fc):
  by_activity_valid=True
  update_display(no_error,display_id="by_activity_check")
  #BY_BTA - MUST BE PROVIDED
  if by_bta==0:
    by_activity_valid=False
    if fc:
      display(bta_error,display_id="by_activity_check")
  return by_activity_valid
by_activity_input=interactive(
    by_activity_check,
    by_bta=by_bta_widget,
    fc=fc_widget)

In [36]:
#USER INPUT FORM - TARGET YEAR ACTIVITY

def ty_activity_check(ty_bta,
                      fc):
  ty_activity_valid=True
  update_display(no_error,display_id="ty_activity_check")
  #TY_BTA - MUST BE PROVIDED
  if ty_bta==0:
    ty_activity_valid=False
    if fc:
      display(bta_error,display_id="ty_activity_check")
  return ty_activity_valid
ty_activity_input=interactive(
    ty_activity_check,
    ty_bta=ty_bta_widget,
    fc=fc_widget)

# TARGET MODEL (DATA & PARAMETERS)

In [37]:
#DEFINE PARAMETERS

pm={#ANNUAL CONTRACTION RATE IN %
    "acr":0.025,
    #YEAR AXIS
    "year":pd.Series(range(2015,2051)).set_axis(range(2015,2051))}

# TARGET MODEL (FUNCTIONS)

In [38]:
#FUNCTION - CHECK IF USER INPUTS SATISFY IMPOSED CONDITIONS

def user_inputs_check():
  if not basic_info_input.result \
  or not by_emissions_input.result \
  or not by_activity_input.result \
  or not ty_activity_input.result:
    user_inputs_valid=False
  else:
    user_inputs_valid=True
  return user_inputs_valid

In [39]:
#FUNCTION - CAPTURE USER INPUTS INTO DICTIONARY

def user_inputs_capture():
  return {"by":basic_info_input.children[0].value,
          "ty":basic_info_input.children[1].value,
          "bte":by_emissions_input.children[0].value,
          "by_bta":by_activity_input.children[0].value,
          "ty_bta":ty_activity_input.children[0].value}

In [40]:
#FUNCTION - ABSOLUTE CONTRACTION APPROACH

def aca(
    #ANNUAL CONTRACTION RATE IN %
    acr=None,
    #BASE YEAR
    by=None,
    #BASE YEAR EMISSIONS
    bye=None,
    #ACTIVITY FORECAST
    af=None):

  #COMPANY ACTIVITY
  output=af
  output=output.to_frame().transpose()
  output.index=["Company Activity"]

  #COMPANY EMISSIONS
  output.loc["Company Emissions"]=bye*(1-(output.columns-by)*acr)

  #COMPANY INTENSITY
  output.loc["Company Intensity"]=output.loc["Company Emissions"]
  output.loc["Company Intensity"]/=output.loc["Company Activity"]

  #MASK YEAR<BASE YEAR
  output.loc[:,output.columns<by]=np.nan

  #SORT
  output=output.sort_index()

  return output

In [41]:
#FUNCTION - TARGET MODEL

def target_model():
  #COMPUTE CAGR FROM BASE YEAR & TARGET YEAR ACTIVITY
  cagr=(ip["ty_bta"]/ip["by_bta"])**(1/(ip["ty"]-ip["by"]))-1

  #ACTIVITY FORECAST
  af=ip["by_bta"]*(1+cagr)**(pm["year"].astype(float)-ip["by"])

  #ACA TARGET
  output=aca(by=ip["by"],bye=ip["bte"],af=af,acr=pm["acr"])

  return output

# RESULTS (FUNCTIONS)

In [42]:
#FUNCTION USED IN COMPANY_TARGET_PLOT

#LABEL BASE YEAR & TARGET YEAR
def label_by_ty(series=None,ax=None):
  x=ip["by"]
  y=series[ip["by"]]
  ax.scatter(x,y,color="blue")
  ax.text(x,y," Base Year",color="blue",verticalalignment="bottom")
  x=ip["ty"]
  y=series[ip["ty"]]
  ax.scatter(x,y,color="blue")
  ax.text(x,y," Target Year",color="blue",verticalalignment="bottom")

In [43]:
#FUNCTION - COMPANY TARGET PLOT

def company_target_plot():
  #FETCH DATA (EXCLUDE YEAR<BASE YEAR)
  chart=target.iloc[:,target.columns>=ip["by"]]

  #SETUP FIGURE
  fig,(ax1,ax2)=plt.subplots(1,2,gridspec_kw={"wspace":0.3})
  fig.set_size_inches(10,4)

  #COMPANY EMISSIONS PLOT
  ax1.plot(chart.loc["Company Emissions"],
          color="blue",
          label="Company Emissions")
  #LABEL BASE YEAR & TARGET YEAR
  label_by_ty(series=chart.loc["Company Emissions"],ax=ax1)
  #FORMAT
  ax1.grid()
  ax1.legend()
  ax1.set_title("Company Emissions Target")
  ax1.set_xlabel("Year")
  ax1.set_ylabel("Emissions (tCO2e)")

  #COMPANY INTENSITY PLOT
  ax2.plot(chart.loc["Company Intensity"],
          color="blue",
          label="Company Intensity")
  #LABEL BASE YEAR & TARGET YEAR
  label_by_ty(series=chart.loc["Company Intensity"],ax=ax2)
  #FORMAT
  ax2.grid()
  ax2.legend()
  ax2.set_title("Company Intensity Target")
  ax2.set_xlabel("Year")
  ax2.set_ylabel("Intensity (tCO2e/FTE)")

  plt.show(block=False)

In [44]:
#FUNCTION - TARGET TABLE

def target_table():
  #FETCH BASE YEAR & TARGET YEAR DATA
  table=target.loc[["Company Intensity",
                    "Company Emissions"],
                     [ip["by"],ip["ty"]]]

  #% REDUCTION
  table["% Reduction ("+str(ip["by"])+"-"+str(ip["ty"])+")"]=\
  -1e2*(table[ip["ty"]]/table[ip["by"]]-1)

  #RENAME ROWS
  table.index=["Company Intensity (tCO2e/FTE)",
               "Company Emissions (tCO2e)"]

  #RENAME COLUMNS
  table=table.rename({ip["by"]:"Base Year ("+str(ip["by"])+")",
                      ip["ty"]:"Target Year ("+str(ip["ty"])+")"},
                     axis=1)

  #ROUNDING
  table.iloc[:,:2]=table.iloc[:,:2].apply(lambda x:round(x,2))
  table.iloc[:,2:]=table.iloc[:,2:].apply(lambda x:round(x))

  #PERCENTAGE SIGN
  table.iloc[:,2]=table.iloc[:,2].astype(int).astype(str)+"%"

  #FORMAT DISPLAY
  table=table.style
  #NUMBER OF DECIMAL PLACES
  table=table.format(precision=2,
                     subset=("Company Intensity (tCO2e/FTE)",slice(None)))
  table=table.format(precision=0,
                     subset=("Company Emissions (tCO2e)",slice(None)))
  #CENTRE ALIGN
  table=table.set_properties(**{"text-align":"center"})
  #COLUMN HEADING - MAX WIDTH 100PX & CENTRE ALIGN
  #ROW HEADING - RIGHT ALIGN
  table=table.set_table_styles(
      [dict(selector=".col_heading",
            props=[("max-width","100px"),("text-align","center")]),
       dict(selector=".row_heading",
            props=[("text-align","right")])])

  display(table)

# FINAL PRODUCT

In [46]:
#BUTTON WIDGET
calculate_target_button=widgets.Button(description="Calculate Target")

#OUTPUT WIDGET
results=widgets.Output()

#SPACER WIDGET
spacer=widgets.HTML(value="<br>")

#TARGET MODEL & RESULTS
@results.capture(clear_output=True)
def target_model_results(_):
  #REGISTER FIRST CLICK
  fc_widget.value=True
  #USER INPUTS DO NOT MEET IMPOSED CONDITIONS
  if not user_inputs_check():
    #OUTPUT ERROR MESSAGE
    display(user_inputs_error)
  #USER INPUTS MEET IMPOSED CONDITIONS
  else:
    #TARGET MODEL
    global ip,target
    ip=user_inputs_capture()
    target=target_model()
    #OUTPUT RESULTS
    display(results_header)
    display(spacer)
    company_target_plot()
    display(spacer)
    target_table()

#LINK BUTTON TO FUNCTION
calculate_target_button.on_click(target_model_results)

#DISPLAY
_=VBox([
    #USER INPUT FORM
    overall_header,
    basic_info_header,
    basic_info_input,
    by_emissions_header,
    by_emissions_input,
    by_activity_header,
    by_activity_input,
    ty_activity_header,
    ty_activity_input,
    ty_activity_footer,
    #TARGET MODEL & RESULTS
    calculate_target_button,
    results],
    layout={"width":"1110px"})
display(_)